In [3]:
import numpy as np
import matplotlib.pyplot as plt
import timm
import numpy as np
import torch.nn as nn
import torch
from torchvision import datasets, models, transforms
from typing import Any
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn import metrics
import gc
import os 
import torch
import torch.nn as nn
import sys
import pandas as pd
import wandb

from einops import rearrange
from einops.layers.torch import Rearrange
import PIL.Image as Image
import PIL as pil
import time
import einops

In [55]:
class restruct(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        
    def forward(self, img):
        """
        Args:
            img (Tensor): The stacked Image .
        Returns:
            Tensor: Restructured Image into 8 channels.
        """
    
        return einops.rearrange(torch.squeeze(img), 'h ( w c ) -> c h w ', w = 125, c=13)


def metric(y_true, y_pred):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    auc = metrics.auc(fpr, tpr)
    return auc

def straightner(a):
    A = np.zeros((a[0].shape[0]*len(a)))
    start_index = 0
    end_index = 0
    for i in range(len(a)):
        start_index = i*a[0].shape[0]
        end_index = start_index+a[0].shape[0]
        A[start_index:end_index] = a[i]
    return A

def predictor(outputs):
    return np.argmax(outputs, axis = 1)


def pil_loader(path: str) -> Image.Image:
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('L')


In [56]:
train_transform = transforms.Compose([
                            transforms.ToTensor(),
                            restruct(),
                            transforms.RandomHorizontalFlip(),
                            transforms.RandomVerticalFlip(),
                            transforms.RandomRotation(60),])



test_transform = transforms.Compose([
                            transforms.ToTensor(),
                            restruct()])


dataset_train = datasets.ImageFolder("/pscratch/sd/b/bbbam/datasetTau_13channels/Train",
                                    transform =train_transform,
                                    loader = pil_loader)

dataset_test = datasets.ImageFolder("/pscratch/sd/b/bbbam/datasetTau_13channels/Test",
                                    transform =test_transform,
                                    loader = pil_loader)

dataloader_test = torch.utils.data.DataLoader(dataset_test,
                                            batch_size=256,
                                            shuffle=True,
                                            drop_last = True,
                                            num_workers=8,
                                            pin_memory = True)

dataloader_train = torch.utils.data.DataLoader(dataset_train,
                                            batch_size=256,
                                            shuffle=True,
                                            drop_last = True,
                                            num_workers=8,
                                            pin_memory = True)


In [62]:
image_size = (128,128)
in_channels = 13
num_blocks = [2, 2, 3, 5, 2]
channels = [64, 96, 192, 384, 768]
num_classes = 1
model = timm.create_model("efficientnet_b0",num_classes=1)
model.conv_stem = nn.Conv2d(in_channels,
                            model.conv_stem.out_channels,
                            model.conv_stem.kernel_size,
                            model.conv_stem.stride,
                            model.conv_stem.padding,
                            bias = False
                           )




optimizer = torch.optim.AdamW(model.parameters(), lr = 0.001, weight_decay = 0.05)
criterion = nn.BCEWithLogitsLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', verbose = True,threshold = 0.0001,patience = 3, factor = 0.5)

model = model.to("cuda:0")



wandb.login(key="51b58a76963008d6010f73edbd6d0617a772c9df")
wandb.init(
    project = "Boosted Tau Classifier",
    name = "Eff-0"
)


scaler = torch.cuda.amp.GradScaler()
#--------------------------
wandb.watch(model, log_freq=500)
#---------------------------
w_intr = 500

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [63]:
for epoch in range(0,10,1):
        train_loss = 0
        val_loss = 0
        train_steps = 0
        test_steps = 0
        label_list = []
        outputs_list = []
        train_auc = 0
        test_auc = 0
        model.train()
        for image, label in tqdm(dataloader_train):
            image = image.to("cuda:0")
            label = label.to("cuda:0")
            with torch.no_grad():
                image = nn.functional.pad(image, (2,1,2,1))

            #optimizer.zero_grad()
            for param in model.parameters():
                param.grad = None

            with torch.cuda.amp.autocast():
                outputs = model(image).squeeze()
                loss = criterion(outputs, label.float())
            label_list.append(label.detach().cpu().numpy())
            outputs_list.append(outputs.detach().cpu().numpy())
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            train_loss += loss.item()
            train_steps += 1
            if train_steps%w_intr == 0:
                    wandb.log({"loss": loss.item()})
        with torch.no_grad():
            label_list = straightner(label_list)
            outputs_list = straightner(outputs_list)
            train_auc = metric(label_list, outputs_list)




        #-------------------------------------------------------------------
        model.eval()
        label_list = []
        outputs_list = []
        with torch.no_grad():
            for image, label in tqdm(dataloader_test):
                image = image.to("cuda:3")
                image = nn.functional.pad(image, (2,1,2,1))
                label = label.to("cuda:3")
                outputs = model(image).squeeze()
                loss = criterion(outputs, label.float())
                label_list.append(label.detach().cpu().numpy())
                outputs_list.append(outputs.detach().cpu().numpy())
                val_loss += loss.item()
                test_steps +=1
                if test_steps%w_intr == 0:
                    wandb.log({"val_loss": loss.item()})
            label_list = straightner(label_list)
            outputs_list = straightner(outputs_list)
            test_auc = metric(label_list, outputs_list)

        train_loss = train_loss/train_steps
        val_loss = val_loss/ test_steps
    #     hist_loss_train.append(train_loss)
    #     hist_loss_test.append(val_loss)
    #     hist_auc_train.append(train_auc)
    #     hist_auc_test.append(test_auc)

        print("----------------------------------------------------")
        print("Epoch No" , epoch)
        print("The Training loss of the epoch, ",train_loss)
        print("The Training AUC of the epoch,  %.3f"%train_auc)
        print("The validation loss of the epoch, ",val_loss)
        print("The validation AUC of the epoch, %.3f"%test_auc)
        print("----------------------------------------------------")
#         PATH = f"/pscratch/sd/t/train130/dc250601/{Run_Name}/model_Epoch_{epoch}.pt"
#         torch.save({
#                 'epoch': epoch,
#                 'model_state_dict': model.state_dict(),
#                 'optimizer_state_dict': optimizer.state_dict(),
#                 'scheduler': scheduler.state_dict()
#                 }, PATH)
        scheduler.step(test_auc)
        curr_lr = scheduler._last_lr[0]
        wandb.log({"Train_auc_epoch": train_auc,
                    "Epoch": epoch,
                    "Val_auc_epoch": test_auc,
                    "Train_loss_epoch": train_loss,
                    "Val_loss_epoch": val_loss,
                    "Lr": curr_lr}
                    )
#         df = pd.read_csv(os.path.join(path,str(Run_Name)+".csv"))
#         df["Present_Lr"][0] = curr_lr
#         df["Present_Wd"][0]  = Present_Wd
#         df["Present_Epoch"][0] = epoch + 1
#         df["New"] = [0]
#         df.to_csv(os.path.join(path,str(Run_Name)+".csv"))
        gc.collect()

wandb.finish()


  0%|          | 0/21093 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f25e6f06dc0>
Traceback (most recent call last):
  File "/global/homes/b/bbbam/.conda/envs/Pytorch_VEN/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/global/homes/b/bbbam/.conda/envs/Pytorch_VEN/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
    if w.is_alive():
  File "/global/homes/b/bbbam/.conda/envs/Pytorch_VEN/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f25e6f06dc0>
Traceback (most recent call last):
  File "/global/homes/b/bbbam/.conda/envs/Pytorch_VEN/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/

KeyboardInterrupt: 

In [64]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


loss,█▇▆▂▃▄▃▂▂▃▂▁▂▂▁▁▁
loss,0.25162
